# 03_Linear_Regression_Models
In this notebook, we will see how to define simple linear regression models.

In [ ]:
import torch
from torch import nn # Basic Neural Network Module

In [ ]:
# Needs for Ploting
import matplotlib.pyplot as plt
%matplotlib inline
torch.manual_seed(1)

## Prepare Data

In [ ]:
# Toy dataset
x_train = torch.tensor([[3.3], [4.4], [5.5], [6.71], [6.93], [4.168], 
                        [9.779], [6.182], [7.59], [2.167], [7.042], 
                        [10.791], [5.313], [7.997], [3.1]])

y_train = torch.tensor([[1.7], [2.76], [2.09], [3.19], [1.694], [1.573], 
                        [3.366], [2.596], [2.53], [1.221], [2.827], 
                        [3.465], [1.65], [2.904], [1.3]])

plt.scatter(x_train.numpy(), y_train.numpy())
plt.show()

## Linear Regression Models
The simplest linear model for regression is one that involves a linear combination of the input variables
$\mathbf{y}^{T} = \mathbf{x}^{T}\mathbf{W} + \mathbf{b}$, $\mathbf{x} \in \mathbb{R}^{n}, \mathbf{y} \in \mathbb{R}^m, \mathbf{W}^{n \times m}, \mathbf{b}^{m}$.


### Define Linear Regression Models
In PyTorch, you can define your weight matrix for linear regression in the following way:

In [ ]:
# Hyper-parameters
input_size = 1
output_size = 1

# Weight Matrix
W = torch.rand(input_size, output_size)
# bias vector
b = torch.zeros(output_size)

You can compute the linear regression ouput using the above weight matrix:

In [ ]:
def myLinear(input):
    return input.mm(W) + b

myLinear(x_train).size()

### Loss Function and Optimization
A loss function takes the (output, target) pair, and computes a value that estimates how far away the output is from the target.

There are several different loss functions for linear regression, but we use the Mean Squared Error:

In [ ]:
# Loss
def myMSE(output, target):
    return torch.mean((output-target)**2)

Furtheremore, we use the Gradient Descent method for optimization.
To do so, we compute the gradient of MSE with respect to the weight matrix $\mathbf{W}$ and the bias vector $\mathbf{b}$:

In [ ]:
# Optimization
def myGradientW(input, output, target):
    return 2 * (output-target).view(-1).dot(input.view(-1)) / len(output)

def myGradientb(output, target):
    return torch.mean(2 * (output-target))

### Training Linear Regression Model

In [ ]:
learning_rate = 0.001
num_epochs = 60
# Train the model
for epoch in range(num_epochs):
    # Compute the prediction(output) of your linear regression model
    output = myLinear(x_train)
    loss = myMSE(output, y_train)
    
    # Compute the gradient w.r.t W and b, respectively.
    gradientW = myGradientW(x_train, output, y_train)
    gradientb = myGradientb(output, y_train)
    
    # Stochastic Gradient Descent
    W -= learning_rate * gradientW
    b -= learning_rate * gradientb
    
    if (epoch+1) % 5 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

### Prediction (Test)

In [ ]:
# Plot the graph
predicted = myLinear(x_train).detach().numpy()
plt.plot(x_train.numpy(), y_train.numpy(), 'ro', label='Original data')
plt.plot(x_train.numpy(), predicted, label='Fitted line')
plt.legend()
plt.show()

### Linear Regression Module with PyTorch Libraries

In fact, PyTorch supports all the above functions in the `torch.nn` package and `torch.optim` package.

In [ ]:
# Hyper-parameters
input_size = 1
output_size = 1
learning_rate = 0.001
num_epochs = 60

# Linear regression model
model = nn.Linear(input_size, output_size)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):

    # Forward pass
    output = model(x_train)
    loss = criterion(output, y_train)
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 5 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [ ]:
# Plot the graph
predicted = model(x_train).detach().numpy()
plt.plot(x_train.numpy(), y_train.numpy(), 'ro', label='Original data')
plt.plot(x_train.numpy(), predicted, label='Fitted line')
plt.legend()
plt.show()

## Training Visualization

In [ ]:
import imageio
import numpy as np

torch.manual_seed(1)    # reproducible

# view data
plt.figure(figsize=(10,4))
plt.scatter(x_train.data.numpy(), y_train.data.numpy(), color = "blue")
plt.title('Regression Analysis')
plt.xlabel('Independent varible')
plt.ylabel('Dependent varible')
plt.show()

In [ ]:
# This code is from http://bit.ly/2YS8qg2

# Linear regression model
model = nn.Linear(input_size, output_size)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


my_images = []
fig, ax = plt.subplots(figsize=(12,7))

# train the network
for t in range(num_epochs):
  
    prediction = model(x_train)     # input x and predict based on x

    loss = criterion(prediction, y_train)     # must be (1. nn output, 2. target)

    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()        # apply gradients
    
    # plot and show learning process
    plt.cla()
    ax.set_title('Regression Analysis', fontsize=35)
    ax.set_xlabel('Independent variable', fontsize=24)
    ax.set_ylabel('Dependent variable', fontsize=24)
    ax.set_xlim(2.0, 11.0)
    ax.set_ylim(1.0, 3.8)
    ax.scatter(x_train.detach().numpy(), y_train.detach().numpy(), color = "blue")
    ax.plot(x_train.detach().numpy(), prediction.detach().numpy(), 'g-', lw=3)
    ax.text(9.0, 1.4, 'Step = %d' % t, fontdict={'size': 24, 'color':  'red'})
    ax.text(9.0, 1.2, 'Loss = %.4f' % loss.detach().numpy(),
            fontdict={'size': 24, 'color':  'red'})

    # Used to return the plot as an image array 
    # (https://ndres.me/post/matplotlib-animated-gifs-easily/)
    fig.canvas.draw()       # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    my_images.append(image)
    
   
# save images as a gif    
imageio.mimsave('./line_1.gif', my_images, fps=10)